<a href="https://colab.research.google.com/github/Kibmor/Ramil_Yarullin_data_analyst/blob/main/12_3_SQL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных:**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**ЗАДАНИЕ:**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
from IPython.display import Markdown
import nbformat
import re
import gdown
import pandas as pd
from sqlalchemy import text,create_engine

## Оглавление

In [2]:
def generate_table_of_contents(notebook_path):
    notebook_url = 'https://drive.google.com/uc?id=' + notebook_path.split('/')[-2]
    gdown.download(notebook_url, 'notebook.ipynb', quiet=True)

    with open('notebook.ipynb', 'r', encoding='utf-8') as file:
        notebook = nbformat.read(file, nbformat.NO_CONVERT)

    table_of_contents = []
    heading_numbers = [0, 0, 0, 0, 0, 0]  # Начальные значения номеров для разных уровней заголовков

    for cell in notebook['cells']:
        if cell['cell_type'] == 'markdown':
            source = cell['source']
            headers = re.findall(r'^(#+)\s+(.*)$', source, flags=re.MULTILINE)

            for header in headers:
                level = len(header[0])
                title = header[1].strip()

                link = '-'.join(re.findall(r'\w+', title.lower()))
                indentation = '&nbsp;' * 2 * (level - 1)
                line_break = '\n' + '&nbsp;' * 2 * level

                heading_numbers[level - 1] += 1
                heading_number_str = '.'.join(str(num) for num in heading_numbers[:level])

                colab_link = notebook_path + f'#scrollTo={link}'
                table_of_contents.append(f'{line_break}- {heading_number_str} {title}')

    markdown_output= '\n'.join(table_of_contents)
    display(Markdown(markdown_output))


# Укажите ссылку на ваш файл на Google Диске
notebook_path = 'https://drive.google.com/file/d/1IA889g40TWgDYwNn2plorEhdFIDIBowO/view?usp=sharing'

# Генерирование оглавления
table_of_contents = generate_table_of_contents(notebook_path)
# Отображение оглавления


&nbsp;&nbsp;- 1 SQL

&nbsp;&nbsp;&nbsp;&nbsp;- 1.1 Оглавление

&nbsp;&nbsp;&nbsp;&nbsp;- 1.2 Выведем датасеты books, authors, publishers, ratings и reviews:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- 1.2.1 1) books:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- 1.2.2 2) authors:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- 1.2.3 3) publishers:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- 1.2.4 4) ratings:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- 1.2.5 5) reviews:

&nbsp;&nbsp;&nbsp;&nbsp;- 1.3 Задание 1:

&nbsp;&nbsp;&nbsp;&nbsp;- 1.4 Задание 2:

&nbsp;&nbsp;&nbsp;&nbsp;- 1.5 Задание 3

&nbsp;&nbsp;&nbsp;&nbsp;- 1.6 Задание 4

&nbsp;&nbsp;&nbsp;&nbsp;- 1.7 Задание 5

&nbsp;&nbsp;&nbsp;&nbsp;- 1.8 Общий вывод:

In [ ]:

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Напишем функцию для вывода результатов запроса:**

In [ ]:
def query(sql):
  return pd.io.sql.read_sql(sql=text(sql),con = engine.connect())

## Выведем датасеты books, authors, publishers, ratings и reviews:

### 1) books:

In [ ]:
books = '''
SELECT *
FROM books

LIMIT 5
'''
query(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем общую информацию:

In [ ]:
print('В датасете books -',query(books).shape[0],'строк и',
      query(books).shape[1],'полей.')

В датасете books - 5 строк и 6 полей.


In [ ]:
query(books).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           5 non-null      int64 
 1   author_id         5 non-null      int64 
 2   title             5 non-null      object
 3   num_pages         5 non-null      int64 
 4   publication_date  5 non-null      object
 5   publisher_id      5 non-null      int64 
dtypes: int64(4), object(2)
memory usage: 368.0+ bytes


___
Количество пропусков:

In [ ]:
query(books).isnull().sum()

book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

___
Количество дубликатов:

In [ ]:
query(books).duplicated().sum()

0

**Вывод:** В таблице books 1000 строк и 6 столбцов, отсутствуют пропуски и дубликаты;

___
### 2) authors:

In [ ]:
authors = '''
SELECT *
FROM authors

LIMIT 5
'''

query(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [ ]:
print('В датасете authors -',query(authors).shape[0],'строк и',
      query(authors).shape[1],'полей.')

В датасете authors - 5 строк и 2 полей.


In [ ]:
query(authors).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  5 non-null      int64 
 1   author     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [ ]:
query(authors).isna().sum()

author_id    0
author       0
dtype: int64

In [ ]:
query(authors).duplicated().sum()

0

**Вывод:** В таблице authors 636 строк и 2 столбца, отсутствуют пропуски и дубликаты;

___
### 3) publishers:

In [ ]:
publishers = '''
SELECT *
FROM publishers

LIMIT 5
'''

query(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [ ]:
print('В датасете publishers -',query(publishers).shape[0],'строк и',
      query(publishers).shape[1],'полей.')

В датасете publishers - 5 строк и 2 полей.


In [ ]:
query(publishers).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  5 non-null      int64 
 1   publisher     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [ ]:
query(publishers).isna().sum()

publisher_id    0
publisher       0
dtype: int64

In [ ]:
query(publishers).duplicated().sum()

0

**Вывод:** В таблице publishers 340 строк и 2 столбца, отсутствуют пропуски и дубликаты;

___
### 4) ratings:

In [ ]:
ratings = '''
SELECT *
FROM ratings

LIMIT 5
'''

query(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [ ]:
print('В датасете ratings -',query(ratings).shape[0],'строк и',
      query(ratings).shape[1],'полей.')

В датасете ratings - 5 строк и 4 полей.


In [ ]:
query(ratings).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  5 non-null      int64 
 1   book_id    5 non-null      int64 
 2   username   5 non-null      object
 3   rating     5 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 288.0+ bytes


In [ ]:
query(ratings).isna().sum()

rating_id    0
book_id      0
username     0
rating       0
dtype: int64

In [ ]:
query(ratings).duplicated().sum()

0

**Вывод:** В таблице ratings 6456 строк и 4 столбца, отсутствуют пропуски и дубликаты;

___
### 5) reviews:

In [ ]:
reviews = '''
SELECT *
FROM reviews

LIMIT 5
'''

query(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [ ]:
query(reviews).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  5 non-null      int64 
 1   book_id    5 non-null      int64 
 2   username   5 non-null      object
 3   text       5 non-null      object
dtypes: int64(2), object(2)
memory usage: 288.0+ bytes


In [ ]:
query(reviews).isna().sum()

review_id    0
book_id      0
username     0
text         0
dtype: int64

In [ ]:
query(reviews).duplicated().sum()

0

**Вывод:** В таблице reviews 2793 строк и 4 столбца, отсутствуют пропуски и дубликаты.

Доступ к базе получили, на таблицы посмотрели, переходим к основной работе


___
___
## Задание 1:

**Посчитаем сколько книг, выпущено после 1 января 2000 года:**

In [ ]:
books_count_after_2000 = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
query(books_count_after_2000)

,count
0,819


**Вывод:** 819 книг было выпущено после 1 января 2000 года;

___
___
## Задание 2:

**Посчитаем для каждой книги количество обзоров и среднюю оценку**

In [ ]:
book_reviews_count__avg_rating = '''
SELECT books.title,
        books.book_id,
        COUNT(DISTINCT reviews.text) AS count_reviews,
        AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id

GROUP BY books.title, books.book_id
ORDER BY count_reviews DESC

'''

query(book_reviews_count__avg_rating)

,title,book_id,count_reviews,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,Water for Elephants,963,6,3.977273
2,The Glass Castle,734,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667
996,The Natural Way to Draw,808,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
998,Essential Tales and Poems,221,0,4.000000


**Вывод:**
Максимальное количество обзоров у книги **Twilight #1 - 7**, затем следует группа книг с количеством обзоров - 6, средняя оценка книг примерно 3.8 - 4.2.

___



___


## Задание 3

**Определим издательство, которое издало наибольшее число книг толще 50 страниц**

In [ ]:
book_publisher_many_pages = '''
SELECT publishers.publisher AS name,
        COUNT(books.book_id) AS count_books

FROM publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id

WHERE books.num_pages > 50

GROUP BY name
ORDER BY count_books DESC
LIMIT 10
'''

query(book_publisher_many_pages)

,name,count_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


**Вывод:** Penguin Books - выпустило наибольшее число книг в количество 42 книг, толще 50 страниц. На 2 и 3 местах издательства Vintage - 31 и Grand Central Publishing - 25.

___
___
## Задание 4

**Определим автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками).**

In [ ]:
high_avg_rating_author = '''

SELECT  books_rating_count_author.author AS author,
        AVG(books_rating_count_author.avg_rating) AS avg_rating

FROM (  SELECT authors.author AS author,
                books.book_id AS id,
                AVG(ratings.rating) AS avg_rating,
                COUNT(ratings.rating) AS count_rating
        FROM authors
        INNER JOIN books ON books.author_id = authors.author_id
        INNER JOIN ratings ON ratings.book_id = books.book_id

        GROUP BY author, id
        HAVING COUNT(ratings.rating) >= 50
      )
            AS books_rating_count_author


GROUP BY author
ORDER BY avg_rating DESC
'''

query(high_avg_rating_author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


**Вывод:** Cамая высокая средняя оценка у автора **J.K. Rowling/Mary GrandPré - 4.284**, далее Markus Zusak/Cao Xuân Việt Khương - 4.264 и J.R.R. Tolkien - 4.258

___
___
## Задание 5

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [ ]:
avg_review_count_with_many_rating_count = '''
SELECT COUNT(*)/COUNT(DISTINCT username) as avg_num_reviews
FROM reviews
WHERE username IN (SELECT username
                    FROM ratings
                    GROUP BY username
                    having COUNT(rating_id)>50
                   )
'''

query(avg_review_count_with_many_rating_count)

,avg_num_reviews
0,24


Среднее кол-во обзоров пользователей, которые поставили более чем по 50 оценок равно 24.

## Общий вывод:

- В таблице book_publisher лежит список всех издательств, которые выпускают книги от 50 страниц;
- В таблице best_author лежит список авторов с самой выской средней оценкой, так как у этих авторов учтены книги которые имеют больше 50 пользовательских оценок;
- В таблице book_reviews есть книги со средней оценкой и пользовательскими обзорами;
- Так же среднее количество текстовых обзоров на книги более чем 50 оценками имеют в среднем 24 текстовых обзора, что будет достаточно информативно для будущих пользователей.